# Linear Probe Experiments
A first try at generating data in a procgen maze environment, and using that to construct a linear probe to detect a "cheese signal" in the neural network layers of a pre-trained RL agent. 

## Initialize Environment 

In [4]:
import torch
import numpy as np
from tqdm import tqdm
from procgen import ProcgenGym3Env
from procgen_tools import maze
from procgen_tools.models import load_policy
from procgen_tools import maze as maze_api

The gym environment will be created through `procgen-tools`. It provides a wrapper around the original procgen environment to make it compatible with gym. 

In [5]:
seed = 4242
wrapped_venv = maze_api.create_venv(
    num=1, start_level=int(seed), num_levels=0, num_threads = 4
)
wrapped_venv.env.get_info()[0].keys()

dict_keys(['prev_level_seed', 'prev_level_complete', 'level_seed', 'rgb'])

In [6]:
# Get the procgen env directly like so
wrapped_venv.env

<CEnv lib_path=/home/gearspark/Projects/ai-safety-camp-2024-model-agents/venv/lib/python3.10/site-packages/procgen/data/prebuilt/libenv.so options={'center_agent': True, 'use_generated_assets': False, 'use_monochrome_assets': False, 'restrict_themes': False, 'use_backgrounds': True, 'paint_vel_info': False, 'distribution_mode': 1, 'env_name': 'maze', 'num_levels': 0, 'start_level': 4242, 'num_actions': 15, 'use_sequential_levels': False, 'debug_mode': 0, 'rand_seed': 149530471, 'num_threads': 4, 'render_human': True, 'resource_root': '/home/gearspark/Projects/ai-safety-camp-2024-model-agents/venv/lib/python3.10/site-packages/procgen/data/assets/'}>

## Load Agent
Download one of the agents from the [trained model files](https://drive.google.com/drive/folders/1Ig7bzRlieyYFcdKL_PM-guSWR8WryDOL). I used `maze_I/model_rand_region_5` without knowing about the performance of this agent. Do not forget to rename the model file or change the filename below.

In [7]:
policy = load_policy('model_rand.pth', action_size=15, device=torch.device('cpu'))

## Run Episodes

In [28]:
def dummy_step_log(model, environment, step, act, obs, rew, done, info):
    """Just a dummy log method"""

def dummy_done_log():
    """Just a dummy log method"""


def run_episode(model, maze_environment, argmax=True, max_time_steps=256, on_step=dummy_step_log, on_done=dummy_done_log):
    model.eval()  # Switch off gradient tracking and other training time mechanisms
    obs = maze_environment.reset()

    for step in range(max_time_steps):
        out, _ = model(torch.FloatTensor(obs))
        if argmax:
            act = out.probs.argmax(dim=-1).numpy()
        else:
            act = out.sample().numpy()
        obs, rew, done, info = maze_environment.step(act)
        on_step(model, maze_environment, step, act, obs, rew, done, info)
        if done:
            break
    on_done()


In [29]:
# Run a test episode
run_episode(policy, wrapped_venv)

In [30]:
# Define a custom logging function
class CustomLogger:

    def __init__(self):
        self.observations = []
        self.rewards = []

    def log(self, model, environment, step, act, obs, rew, done, info):
        self.observations.append(obs)
        self.rewards.append(float(rew[0]))

    def reset_logs(self):
        self.observations = []
        self.rewards = []

logger = CustomLogger()
run_episode(policy, wrapped_venv, on_step=logger.log)

In [31]:
obs = logger.observations
rews = logger.rewards
print(f"n obs: {len(obs)}, n rewards: {len(rews)}, total reward: {sum(rews)}")

n obs: 244, n rewards: 244, total reward: 0.0


## Logging Model Activations
Activations can be logged with `circrl`. The hook manager logs a single activation by default. Since we want to have activations spread over the entire eposide we will use a custom hook manager that can be reset between episodes.

In [32]:
def get_model_layer_names(model):
    return [name for name, module in model.named_modules()]

In [33]:
names = get_model_layer_names(policy)
print(names[:5])

['', 'embedder', 'embedder.block1', 'embedder.block1.conv', 'embedder.block1.maxpool']


Create a hook to log the activations of the corresponding layers.

In [34]:
from circrl import hooks

policy_hook = hooks.HookManager(
    model=policy,
    cache=names[1:4]
)

print("Before logging")
print(policy_hook.cache_results.keys())

# Run an environment update in the policy hook context to start logging
with policy_hook:
    obs = wrapped_venv.reset()
    output = policy(torch.FloatTensor(obs))

print("After logging")
print(output)
print(policy_hook.cache_results.keys())

Before logging
dict_keys([])
After logging
(Categorical(logits: torch.Size([1, 15])), tensor([8.4563], grad_fn=<ViewBackward0>))
dict_keys(['embedder.block1.conv', 'embedder.block1', 'embedder'])


Be careful! In the default log setting the `cache_results` gets overwritten every time the policy is run:

In [35]:
with policy_hook:
    obs = wrapped_venv.reset()
    output = policy(torch.FloatTensor(obs))

print("log size after a single execution:")
print(f"\t{policy_hook.cache_results[names[1]].size()}")

with policy_hook:    
    obs = wrapped_venv.reset()
    for _ in range(10):
        output = policy(torch.FloatTensor(obs))

print("Log size after a multiple executions:")
print(f"\t{policy_hook.cache_results[names[1]].size()}")

log size after a single execution:
	torch.Size([1, 256])
Log size after a multiple executions:
	torch.Size([1, 256])


## Gathering Training Data
To gather training data we need to run multiple episodes and store the activations of the model alongside the environment parameters. Procgen-tools provides some convenience functions to make this easier.

In [36]:
def generate_maze_training_data(model, n_episodes, on_episode_step, on_episode_done):
    max_seed = int(1e9)
    seeds = np.random.default_rng().choice(max_seed, size=n_episodes, replace=False)

    for seed in tqdm(seeds):
        wrapped_venv = maze_api.create_venv(
            num=1, start_level=int(seed), num_levels=0, num_threads = 4
        )  # Convenience functions
        state = maze_api.state_from_venv(wrapped_venv)
        grid = maze_api.get_grid(state.state_vals)
        cheese_pos = maze_api.get_cheese_pos(grid)

        run_episode(model, wrapped_venv, on_step=on_episode_step, on_done=on_episode_done)

    print(cheese_pos)
    print(len(grid))

In [37]:
generate_maze_training_data(policy, 1, dummy_step_log, dummy_done_log)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.97s/it]

(12, 21)
25


We need a system to log activations and environment variables persistently. It would be best to avoid having to edit code related to model execution when logging to avoid introducting bugs. We can use something like the following logging setup to log episodic data to disk at the end of every episode. It combines the previous logging function with hooks:

In [43]:
import pickle

class ProbeDataLogger:
    def __init__(self, model, save_file_path=""):
        self._init_logs()

        self.model = model
        layer_names = get_model_layer_names(model)
        self.model_hook = hooks.HookManager(
            model=model,
            cache=names
        )
        self._file_name = save_file_path + "probe_logs"
        self._file_extension = ".pkl"
        self._current_ep = 0

    def log(self, model, environment, step, act, obs, rew, done, info):
        self.observations.append(obs)
        self.rewards.append(rew)
        self.activations.append(self.model_hook.cache_results)

    def done(self):
        self._current_ep += 1
        self.flush()

    def flush(self):
        """Flush data in log buffer to file and reset buffer"""
        self._save_logs_to_file(self._file_name + "_" + str(self._current_ep) + self._file_extension)
        self.reset()

    def reset(self):
        self._init_logs()

    def _save_logs_to_file(self, file_path):
        write_mode = 'wb' 
        with open(file_path, write_mode) as file:
            pickle.dump(self.observations, file)
            pickle.dump(self.rewards, file)
            pickle.dump(self.activations, file)

    def _init_logs(self):
        self.observations = []
        self.rewards = []
        self.activations = []

In [44]:
logs = ProbeDataLogger(policy)
with logs.model_hook:
    generate_maze_training_data(policy, 3, logs.log, logs.done)

100%|██████████| 3/3 [00:04<00:00,  1.34s/it]

(18, 6)
25


Now we can load back the data. Including the loading functionality in the logging class might be convenient, but here I will simply recover it manually as an example.

In [48]:
import pickle

def load_activations(file_path):
    """
    Loads the activations from a specified pickle file.
    
    Args:
    - file_path (str): The path to the pickle file.
    
    Returns:
    - activations (list): The list of activations unpickled from the file.
    """
    with open(file_path, 'rb') as file:
        # Unpickle in the order they were saved: observations, rewards, activations
        observations = pickle.load(file)
        rewards = pickle.load(file)
        activations = pickle.load(file)
    
    return activations

# Example usage
file_path = 'probe_logs_1.pkl'
activations = load_activations(file_path)
print("Loaded activations:", len(activations))

Loaded activations: 5


## Training A Linear Probe